In [2]:
import pandas as pd

In [3]:
# Imports data from csv file
df= pd.read_csv('/Users/nieksonneveld/code/nieksonneveld/election-predictor/raw_data/Full Polling Data, Election Vault, 1955-2024.csv')

In [4]:
# Removes spaces and ampersands in pollster names
df['pollster'] = df['pollster'].str.replace(' ', '').str.replace('&', '').str.replace('-', '')

In [5]:
# Converts startdate/enddate to datetime
df['startdate'] = pd.to_datetime(df['startdate'], format='mixed')
df['enddate'] = pd.to_datetime(df['enddate'], format='mixed')

In [6]:
# Creates unique index for each poll
df_uuid = df.set_index(df['enddate'].dt.strftime('%Y-%m-%d').apply(str).str.replace('-', '_') + '_' + df['pollster'])

In [33]:
# Pivots table to create column for each party
all_df = df_uuid.pivot_table(values="votingintention", index=[df_uuid.index,\
                                                                        'startdate', 'enddate', 'pollster', 'samplesize', 'countrycode'], columns=['partycode'])
all_df.reset_index(level=['startdate', 'enddate', 'pollster', 'samplesize', 'countrycode'], inplace=True)


In [34]:
# Countrycode definitions
sco = ['SCO'] # defines Scotland
gb = ['GBR'] # defines Great Britain
wal = ['WAL'] # defines Wales
nir = ['NIR']   # defines Northern Ireland
ukm = ['UKM']  # defines United Kingdom
enw = ['ENW'] # defines England and Wales

In [35]:
# Creates subsets of polls
sco_polls = all_df[all_df['countrycode'].isin(sco)]
gb_polls = all_df[all_df['countrycode'].isin(gb)]
wal_polls = all_df[all_df['countrycode'].isin(wal)]
nir_polls = all_df[all_df['countrycode'].isin(nir)]
ukm_polls = all_df[all_df['countrycode'].isin(ukm)]
enw_polls = all_df[all_df['countrycode'].isin(enw)]

In [36]:
all_df.head()

partycode,startdate,enddate,pollster,samplesize,countrycode,ALL,BRX,CHN,CON,DUP,...,NAT,OTH,PLC,SDL,SDP,SFA,SNP,TUV,UKI,UUP
1943_06_30_Gallup,1943-01-06,1943-06-30,Gallup,1000,GBR,NaN,NaN,NaN,36.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1943_07_31_Gallup,1943-01-07,1943-07-31,Gallup,1000,GBR,NaN,NaN,NaN,33.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1943_12_31_Gallup,1943-01-12,1943-12-31,Gallup,1000,GBR,NaN,NaN,NaN,32.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1944_01_31_Gallup,1944-01-01,1944-01-31,Gallup,1000,GBR,NaN,NaN,NaN,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1945_02_28_Gallup,1945-01-02,1945-02-28,Gallup,1000,GBR,NaN,NaN,NaN,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# remove all data before 2004
df_2004_on = all_df[all_df.enddate > '2003-12-31'].copy()

In [75]:
df_2004_on.shape

(5408, 23)

In [76]:
df_2004_on['election_diff'] = 0
df_2004_on['relevant_election'] = ''

In [77]:
df_2004_on.iloc[np.where(df_2004_on.enddate < '2005-05-05')[0]].index

Index(['2004_01_08_Populus', '2004_01_18_ICM', '2004_02_22_ICM',
       '2004_03_07_Populus', '2004_04_01_Populus', '2004_04_18_ICM',
       '2004_05_09_Populus', '2004_05_12_Populus', '2004_05_23_ICM',
       '2004_06_06_Populus', '2004_06_13_ICM', '2004_06_20_IpsosMORI',
       '2004_07_03_Populus', '2004_07_18_ICM', '2004_08_02_Populus',
       '2004_08_15_ICM', '2004_09_05_Populus', '2004_09_19_ICM',
       '2004_09_23_YouGov', '2004_10_24_ICM', '2004_11_03_ICM',
       '2004_11_14_ICM', '2004_12_19_ICM', '2005_01_23_ICM',
       '2005_02_05_Populus', '2005_02_20_ICM', '2005_03_04_ICM',
       '2005_03_04_Populus', '2005_03_05_ICM', '2005_03_05_Populus',
       '2005_03_11_ICM', '2005_03_20_ICM', '2005_04_05_IpsosMORI',
       '2005_04_09_Populus', '2005_04_17_Populus', '2005_04_19_ICM',
       '2005_04_19_Populus', '2005_04_20_Populus', '2005_04_21_Populus',
       '2005_04_22_Populus', '2005_04_23_Populus', '2005_04_24_Populus',
       '2005_04_25_Populus', '2005_04_26_ICM', '200

In [78]:
import numpy as np
from datetime import datetime
df_2004_on.loc[df_2004_on.iloc[np.where(df_2004_on.enddate < '2005-05-05')[0]].index, 'election_diff'] = (datetime.strptime('2005-05-05', '%Y-%M-%d') - df_2004_on.enddate) / np.timedelta64(1, 'D')

ValueError: cannot reindex on an axis with duplicate labels

In [79]:
df_2004_on

partycode,startdate,enddate,pollster,samplesize,countrycode,ALL,BRX,CHN,CON,DUP,...,PLC,SDL,SDP,SFA,SNP,TUV,UKI,UUP,election_diff,relevant_election
2004_01_08_Populus,2004-07-31,2004-01-08,Populus,570,GBR,NaN,NaN,NaN,32.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
2004_01_18_ICM,2004-01-16,2004-01-18,ICM,1007,GBR,NaN,NaN,NaN,34.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
2004_02_22_ICM,2004-02-20,2004-02-22,ICM,1006,GBR,NaN,NaN,NaN,34.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
2004_03_07_Populus,2004-02-07,2004-03-07,Populus,556,GBR,NaN,NaN,NaN,29.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
2004_04_01_Populus,2004-02-01,2004-04-01,Populus,566,GBR,NaN,NaN,NaN,35.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024_12_02_YouGov,2024-09-02,2024-12-02,YouGov,4041,GBR,NaN,12.0,NaN,22.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
2024_12_04_Omnisis,2024-11-04,2024-12-04,Omnisis,1271,GBR,NaN,11.0,NaN,24.0,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0,
2024_12_04_Panelbase,2024-09-04,2024-12-04,Panelbase,1086,SCO,NaN,5.0,NaN,16.0,NaN,...,NaN,NaN,NaN,NaN,32.0,NaN,NaN,NaN,0,
2024_12_05_RedfieldWilton,2024-12-05,2024-12-05,RedfieldWilton,3000,GBR,NaN,15.0,NaN,21.0,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,0,
